# Импорт библиотек




In [1]:
import pandas as pd
from itertools import combinations
import os

In [2]:
# Загрузка CSV-файла c леммами без предлогов
file_path = os.path.join("..", "data", "csv", "no_prepositions.csv")
df = pd.read_csv(file_path)

## создание таблиц узлов и ребер

In [4]:
# Находим все колонки, содержащие леммы (оканчиваются на '_lemmas')
collections = [col for col in df.columns if col.endswith('_lemmas')]

# Создаем словарь: {название сборника: множество его лемм}
collection_lemmas = {
    col.replace("_lemmas", ""): set(df[col].dropna())
    for col in collections
}

# Подготовка рёбер и атрибутов узлов для бимодальной сети
edges_bimodal = []
nodes_attributes = []

for collection, lemmas in collection_lemmas.items():
    nodes_attributes.append((collection, "collection"))  # сборник как узел
    for lemma in lemmas:
        edges_bimodal.append((collection, lemma))  # ребро
        nodes_attributes.append((lemma, "lemma"))  # лемма как узел

# Удаляем дубликаты узлов
nodes_attributes = list(set(nodes_attributes))

# Папка для вывода
output_dir = os.path.join("..", "output", "for_gephi")
os.makedirs(output_dir, exist_ok=True)

# Папка для файлов с общими леммами
common_lemmas_dir = os.path.join("..", "output", "shared_lemmas_txt")
os.makedirs(common_lemmas_dir, exist_ok=True)

# Сохраняем рёбра и узлы в CSV
edges_bimodal_df = pd.DataFrame(edges_bimodal, columns=["Source", "Target"])
nodes_attributes_df = pd.DataFrame(nodes_attributes, columns=["Id", "Attribute"])

edges_csv_path = os.path.join(output_dir, "bimodal_edges.csv")
nodes_csv_path = os.path.join(output_dir, "bimodal_nodes.csv")

edges_bimodal_df.to_csv(edges_csv_path, index=False)
nodes_attributes_df.to_csv(nodes_csv_path, index=False)

# Сохраняем файлы с общими леммами между парами сборников
for coll1, coll2 in combinations(collection_lemmas.keys(), 2):
    common_lemmas = collection_lemmas[coll1] & collection_lemmas[coll2]
    if common_lemmas:
        file_name = f"{coll1} - {coll2} - общие леммы.txt"
        file_path = os.path.join(common_lemmas_dir, file_name)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write("\n".join(sorted(common_lemmas)))

print("Готово!")
print(f" Файлы 'bimodal_edges.csv' и 'bimodal_nodes.csv' сохранены в {output_dir}")
print(f" Общие леммы по парам сборников — в {common_lemmas_dir}")


Готово!
 Файлы 'bimodal_edges.csv' и 'bimodal_nodes.csv' сохранены в ../output/for_gephi
 Общие леммы по парам сборников — в ../output/shared_lemmas_txt
